## Initial Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [28]:
import pandas as pd
import numpy as np
from classipy import DataFrameTransformer, JSONtoModelTransformer, CustomLabelEncoder, CustomVotingClassifier
from sklearn.base import BaseEstimator
import joblib

#
---

# Frontend

### Read CSV in Dataframe

In [101]:
df_in = pd.read_csv('../raw_data/kaggle_housing.csv')
df_in.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Convert Dataframe to JSON

In [103]:
df_json = DataFrameTransformer(dataset_name='kaggle_housing').fit_transform(df_in)
df_json.head()

Processing DataFrame: kaggle_housing None


,dataset_name,column_name,column_values,column_values_unique,n_unique_values,unique_value_counts,n_values,mean,std,median,skew,kurt,shapiro_wilk_test
0,kaggle_housing,Id,"1385, 354, 176, 278, 1446, 202, 914, 1407, 936...","[1385, 354, 176, 278, 1446, 202, 914, 1407, 93...",1000,"{1385: 1, 444: 1, 1322: 1, 1109: 1, 395: 1, 13...",1000,735.769000,418.350505,747.0,-0.038346,-1.163219,2.164696e-16
1,kaggle_housing,MSSubClass,"60, 50, 50, 60, 190, 20, 20, 60, 70, 20, 60, 6...","[60, 50, 190, 20, 70, 160, 80, 85, 120, 45, 75...",15,"{20: 387, 60: 198, 50: 95, 120: 57, 160: 43, 3...",1000,56.250000,42.755298,50.0,1.434802,1.621111,1.179931e-33
2,kaggle_housing,MSZoning,"RL, RL, RL, RL, RL, RM, RL, RL, RL, RM, RL, RL...","[RL, RM, FV, C (all), RH]",5,"{'RL': 796, 'RM': 141, 'FV': 48, 'C (all)': 8,...",1000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00
3,kaggle_housing,LotFrontage,"21.0, 60.0, 85.0, 54.0, 74.0, 80.0, 89.0, nan,...","[21.0, 60.0, 85.0, 54.0, 74.0, 80.0, 89.0, nan...",102,"{60.0: 99, 80.0: 48, 70.0: 47, 50.0: 37, 65.0:...",1000,70.284848,23.706592,69.0,1.887697,14.596583,1.000000e+00
4,kaggle_housing,LotArea,"12886, 5330, 9591, 14260, 4224, 10240, 10171, ...","[12886, 5330, 9591, 14260, 4224, 10240, 10171,...",759,"{6000: 16, 7200: 15, 9600: 14, 10800: 12, 8400...",1000,10851.117000,11515.224593,9555.0,11.396430,165.488985,0.000000e+00


### Send to Backend

In [33]:
jsonified_df = df_json.to_json()

# Send PostRequest
import requests
# response = requests.post('api_endpoint',{'data':jsonified_df})

# 
---

# Backend

### Read JSON into Dataframe

In [40]:
df = pd.read_json(jsonified_df)
df.head()

,dataset_name,column_name,column_values,column_values_unique,n_unique_values,unique_value_counts,n_values,mean,std,median,skew,kurt,shapiro_wilk_test
0,kaggle_housing,Id,"1237, 1440, 197, 1269, 98, 97, 1092, 186, 1070...","[1237, 1440, 197, 1269, 98, 97, 1092, 186, 107...",1000,"{'1237': 1, '316': 1, '907': 1, '1168': 1, '60...",1000,729.694000,426.068244,729.5,0.004605,-1.240440,9.390852e-18
1,kaggle_housing,MSSubClass,"70, 45, 60, 20, 60, 20, 20, 20, 20, 20, 80, 60...","[70, 45, 60, 20, 80, 120, 90, 30, 160, 50, 75,...",15,"{'20': 371, '60': 210, '50': 95, '120': 57, '3...",1000,56.950000,42.584163,50.0,1.416365,1.591599,2.752833e-33
2,kaggle_housing,MSZoning,"RL, RL, RM, RL, RL, RL, RL, RL, RL, RL, RL, RL...","[RL, RM, FV, C (all), RH]",5,"{'RL': 810, 'RM': 132, 'FV': 43, 'RH': 9, 'C (...",1000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00
3,kaggle_housing,LotFrontage,"81.0, nan, 75.0, nan, 60.0, 41.0, nan, 55.0, 6...","[81.0, None, 75.0, 60.0, 41.0, 55.0, 65.0, 50....",105,"{'60.0': 89, '70.0': 49, '80.0': 45, '50.0': 4...",1000,70.797101,25.066680,70.0,2.680296,21.553154,1.000000e+00
4,kaggle_housing,LotArea,"12474, 3363, 9855, 12546, 7153, 13006, 2280, 9...","[12474, 3363, 9855, 12546, 7153, 13006, 2280, ...",765,"{'9600': 19, '7200': 18, '9000': 11, '6000': 1...",1000,10614.249000,11287.270838,9452.5,11.992856,179.725898,0.000000e+00


### Transform Dataframe to Model Ready Dataframe

In [96]:
X = JSONtoModelTransformer().fit_transform(df)
X.head()

,n_unique_values,n_values,mean,std,median,skew,kurt,shapiro_wilk_test
0,1.000000,0.0,0.004101,0.005616,0.004559,0.033493,0.000135,9.390852e-18
1,0.013026,0.0,0.000320,0.000561,0.000313,0.102279,0.005902,2.752833e-33
2,0.003006,0.0,0.000000,0.000000,0.000000,0.033269,0.002661,0.000000e+00
3,0.103206,0.0,0.000398,0.000330,0.000438,0.163862,0.046547,1.000000e+00
4,0.764529,0.0,0.059652,0.148776,0.059078,0.617600,0.368615,0.000000e+00


### Feed Model Ready Dataframe to Predictor

In [97]:
from classipy import CustomVotingClassifier

cvc = CustomVotingClassifier()

y_preds = cvc.predict(X)

pd.DataFrame(y_preds).head()

,0
0,date
1,date
2,cat-multi
3,date
4,int


### Return Predictions

In [98]:
pd.DataFrame({
    'column_names':df['column_name'],
    'y_preds': y_preds,
})

,column_names,y_preds
0,Id,date
1,MSSubClass,date
2,MSZoning,cat-multi
3,LotFrontage,date
4,LotArea,int
...,...,...
76,MoSold,date
77,YrSold,date
78,SaleType,cat-multi
79,SaleCondition,cat-multi


### Option 1: Return JSON with ONLY predictions

In [ ]:
import json 

json_file_opt1 = json.dumps(y_preds.tolist())
json.loads(json_file_opt1)

### Option 2: Return JSON with ColumnNames and Predictions

In [100]:
json_file = pd.DataFrame({
    'column_names':df['column_name'],
    'y_preds_decoded': y_preds
}).to_json()


# Example how to query on Frontend
pd.read_json(json_file).set_index(['column_names']).loc['Id'].values[0]


'date'